# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook...

Imports and set magics:

In [6]:
import numpy as np

from types import SimpleNamespace
from scipy import optimize
import pandas as pd 
import matplotlib.pyplot as plt
import math

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Question 1

**Explain how you solve the model**

In [2]:
from inauguralproject import HouseholdClass 

In [5]:
model = HouseholdClass()
model.solve_discrete()

TypeError: unsupported operand type(s) for ** or pow(): 'function' and 'float'

In [ ]:
par = model.par
sol = model.sol
print(f'{par.alpha = :.4f}')
print(f'{par.sigma = :.4f}')

In [ ]:
model.solve_discrete()
print(f'(HF) = ({model.solve_discrete.HF:.3f}))

In [ ]:
# code for solving the model (remember documentation and comments)

a = np.array([1,2,3])
b = inauguralproject.square(a)
print(b)

# Question 2

Explain your code and procedure

In [ ]:
# code

# Question 3

Explain your code and procedure

In [ ]:
# code


ADD CONCISE CONLUSION.

# Question 5